In [1]:
import numpy as np
import pandas as pd
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import matplotlib.pyplot as plt
from astropy.table import Column, join, Table, vstack, hstack
from astropy.io import ascii
from HMXB_pipeline_class import pipeline
import os
import pylatex

In [2]:
cwd = os.getcwd()
home_files = os.path.dirname(cwd)
home_files = home_files + '/'

In [3]:
csv_files = home_files + '/DATA/'
csv_files

'/home/karan/Documents/UvA/Thesis//DATA/'

In [4]:
HMXB_table = ascii.read(csv_files+'HMXB_20241212_.csv',format='csv')

In [5]:
HMXB_table

source_id,ra,ra_error,dec,dec_error,pmra,pmra_error,pmdec,pmdec_error,parallax,parallax_error,radial_velocity,radial_velocity_error,phot_g_mean_mag,l,b,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,distance,pm_l_poleski,pm_b_poleski,galactic distance,circular velocity,LSR velocity,mu_l_sol,mu_b_sol,mu_l_rot,mu_b_rot,Peculiar Velocity,peculiar_mu_l,peculiar_mu_b,dist_err,SpType,Mod_SpType,SpColor,Name,Mx,Mx_err,Mo,Mo_err
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str21,str17,str10,str23,float64,float64,float64,float64
3441207615229815040,84.72739245821431,0.020250700414180756,26.31577524440175,0.015788812190294266,-0.5898287895613142,0.030902111902832985,-2.879557993302878,0.01621093973517418,0.5245380785708109,0.023250581696629524,nan,nan,8.601999282836914,181.44504956690048,-2.643537066457817,8.987546920776367,8.01044750213623,0.9770994186401367,1.9064392860183996,2.12603198111763,-2.0297143279424583,10.403915664294958,225.02622744453242,-4.5772627558119305,1.4743856024086786,-0.7832807603717467,-0.0025775251985549665,0.005181971135181485,12.762149422593925,0.6542239039075064,-1.2516155387058931,0.08450448914978252,O9.7IIIe,xkcd:bright blue,xkcd:blue,1A 0535+262,--,--,20.0,--
5336957010898124160,170.23817252151974,0.008774157613515854,-61.91670671055608,0.009544741362333298,-5.421032061547652,0.012438164092600346,1.3700031073878873,0.011767985299229622,0.32925822598102933,0.011463727802038193,nan,nan,11.589363098144531,292.49852714674694,-0.8918067117070271,12.270934104919434,10.759902000427246,1.5110321044921875,3.03712989104672,-5.567951674993604,-0.5122609408455543,7.8560253894545635,222.56671751604506,-220.47262223549708,-1.054611102383597,-0.5187441308440925,-5.428597657240247,-0.018013469887746873,13.18075554666247,0.9152570846302401,0.024496659886285126,0.10574323622942539,O9.5Ve,xkcd:dark blue,xkcd:blue,1A 1118-61,--,--,--,--
5334851450481641088,176.86894042633617,0.007541968487203121,-61.953719225831875,0.008998828940093517,-6.225580251990721,0.01041429303586483,2.361537685037365,0.011920752003788948,0.12669559944899778,0.010234012268483639,nan,nan,12.255293846130371,295.48972809721556,-0.00970401441105653,13.309616088867188,11.227763175964355,2.081852912902832,7.892933964155221,-6.606609744728988,0.8291063820044122,8.763800120419218,223.52091675977692,-193.7528220168817,-0.4170130760677609,-0.20310608662182214,-5.372222021128227,2.7060920841833756e-05,49.25833635059616,-0.8173746475330006,1.0321854077053925,0.6375626570677606,B2Iae,xkcd:dark green,xkcd:green,1E 1145.1-6141,1.7,0.3,14.0,4.0
6053076566300433920,183.31156879290245,0.012357387691736221,-64.87513252303748,0.015193149447441101,-5.95305662663964,0.01588580757379532,0.449629791184431,0.02075292356312275,0.26448691327892004,0.01766935922205448,nan,nan,13.976888656616211,298.89007133029565,-2.303521643360621,14.39801025390625,13.351907730102539,1.046102523803711,3.7809054051208566,-5.958118420264099,-0.37666303137996027,7.449418244846888,222.10444742848472,-219.8306787168525,-0.8942614068632253,-0.40872604268783935,-5.393397656845806,-0.06092405130983405,6.1364723359412885,0.3295406434449326,0.09298706261771314,0.25258783113112304,B5V,xkcd:red,xkcd:red,1ES 1210-64.6,--,--,--,--
5255509901121774976,154.7315592681715,0.00934557057917118,-58.94609420856299,0.009438585489988327,-6.454306520313594,0.013439311645925045,2.256007949459996,0.01335159968584776,0.2273178405234294,0.010232986882328987,nan,nan,12.269230842590332,284.351776705235,-1.6900802660463154,12.882477760314941,11.495985984802246,1.3864917755126953,4.3991267807989365,-6.6488043210162475,-1.5940657536704546,8.547292409868648,223.3022676369862,-212.96405708690497,-0.6634443517918205,-0.34946803248438035,-5.444675837248132,0.0016679885391937266,28.32721701626133,-0.5406841319762945,-1.24626570

In [6]:
HMXB_table_pd = HMXB_table.to_pandas()


In [7]:
output = HMXB_table_pd[['Name','SpType','phot_g_mean_mag','l','b','parallax','parallax_error','distance','dist_err','pm_l_poleski','pm_b_poleski','Peculiar Velocity','Mx','Mo']]

In [8]:
#output = output.round({'l':2,'b':2,'distance':2,'pm_l_poleski':2,'pm_b_poleski':2,'Peculiar_velocity':1})
output = output.round(decimals=3)

# Mask shitting parallax

In [9]:
def make_prlx_mask(table):
    prlx_mask = table['parallax']/table['parallax_error'] >=5.0
    table = table[prlx_mask]
    return table
output = make_prlx_mask(output)

In [10]:

output["Parallax (Error)"] = output.apply(
    lambda row: f"{row['parallax']:.2f}({row['parallax_error']:.4f})", axis=1
)


output["Distance (Error)"] = output.apply(
    lambda row: f"{row['distance']:.2f}({row['dist_err']:.4f})", axis=1
)

output = output.drop(columns=["parallax", "parallax_error"])
output = output.drop(columns=["distance", "dist_err"])

,Name,SpType,phot_g_mean_mag,l,b,Parallax (Error),pm_l_poleski,pm_b_poleski,Peculiar Velocity,Mx,Mo,Distance (Error)
0,1A 0535+262,O9.7IIIe,8.602,181.445,-2.644,0.53(0.0230),2.126,-2.030,12.76,NaN,20.0,1.91(0.0850)
1,1A 1118-61,O9.5Ve,11.589,292.499,-0.892,0.33(0.0110),-5.568,-0.512,13.18,NaN,NaN,3.04(0.1060)
2,1E 1145.1-6141,B2Iae,12.255,295.490,-0.010,0.13(0.0100),-6.607,0.829,49.26,1.7,14.0,7.89(0.6380)
3,1ES 1210-64.6,B5V,13.977,298.890,-2.304,0.26(0.0180),-5.958,-0.377,6.14,NaN,NaN,3.78(0.2530)
4,1FGL J1018.6-5856,O6V((f)),12.269,284.352,-1.690,0.23(0.0100),-6.649,-1.594,28.33,2.0,22.9,4.40(0.1980)
...,...,...,...,...,...,...,...,...,...,...,...,...
108,TYC 3681-695-1,B1-2 III/Ve,11.412,126.080,-3.568,0.34(0.0180),-2.392,-0.801,13.26,NaN,NaN,2.95(0.1570)
110,V0332+53,O8-9Ve,14.200,146.052,-2.194,0.13(0.0200),-0.476,0.198,18.23,NaN,NaN,7.44(1.1160)
111,XTE J0421+560,B1/2I[e],10.766,149.177,4.133,0.21(0.0150),0.027,-0.695,10.76,NaN,NaN,4.76(0.3370)
113,XTE J1739-302,O8Iab(f),12.643,358.068,0.445,0.52(0.0480),2.957,2.361,50.50,NaN,33.7,1.94(0.1800)


In [19]:
columns = output.columns.tolist()
columns.insert(5, columns.pop(columns.index("Parallax (Error)")))
columns.insert(6, columns.pop(columns.index("Distance (Error)")))
output = output[columns]

In [20]:
output

,Name,SpType,phot_g_mean_mag,l,b,Parallax (Error),Distance (Error),pm_l_poleski,pm_b_poleski,Peculiar Velocity,Mx,Mo
0,1A 0535+262,O9.7IIIe,8.602,181.445,-2.644,0.53(0.0230),1.91(0.0850),2.126,-2.030,12.76,NaN,20.0
1,1A 1118-61,O9.5Ve,11.589,292.499,-0.892,0.33(0.0110),3.04(0.1060),-5.568,-0.512,13.18,NaN,NaN
2,1E 1145.1-6141,B2Iae,12.255,295.490,-0.010,0.13(0.0100),7.89(0.6380),-6.607,0.829,49.26,1.7,14.0
3,1ES 1210-64.6,B5V,13.977,298.890,-2.304,0.26(0.0180),3.78(0.2530),-5.958,-0.377,6.14,NaN,NaN
4,1FGL J1018.6-5856,O6V((f)),12.269,284.352,-1.690,0.23(0.0100),4.40(0.1980),-6.649,-1.594,28.33,2.0,22.9
...,...,...,...,...,...,...,...,...,...,...,...,...
108,TYC 3681-695-1,B1-2 III/Ve,11.412,126.080,-3.568,0.34(0.0180),2.95(0.1570),-2.392,-0.801,13.26,NaN,NaN
110,V0332+53,O8-9Ve,14.200,146.052,-2.194,0.13(0.0200),7.44(1.1160),-0.476,0.198,18.23,NaN,NaN
111,XTE J0421+560,B1/2I[e],10.766,149.177,4.133,0.21(0.0150),4.76(0.3370),0.027,-0.695,10.76,NaN,NaN
113,XTE J1739-302,O8Iab(f),12.643,358.068,0.445,0.52(0.0480),1.94(0.1800),2.957,2.361,50.50,NaN,33.7


In [21]:
#above didn't work but this does
output['Peculiar Velocity']=output['Peculiar Velocity'].round(2)


In [14]:
home_files

'/home/karan/Documents/UvA/Thesis/'

In [25]:
headers = ['Name','SpType','G', 'l', 'b', 'Parallax', 'Distance', r'$\mu_l$ cos(b)', r'$\mu_b$', r'$V pec$', 'Mx', 'Mo']
latex_table = output.to_latex(index=False,longtable=True,float_format="%.2f",header=headers)
with open(f"{home_files}/Tables/HMXBstarsv3.tex", "w") as file:
    file.write(latex_table)

In [23]:
latex_table

'\\begin{longtable}{llrrrllrrrrr}\n\\toprule\nName & SpType & G & l & b & Parallax & Distance & $\\mu_l$ cos(b) & $\\mu_b$ & $V pec$ & Mx & Mo \\\\\n\\midrule\n\\endfirsthead\n\\toprule\nName & SpType & G & l & b & Parallax & Distance & $\\mu_l$ cos(b) & $\\mu_b$ & $V pec$ & Mx & Mo \\\\\n\\midrule\n\\endhead\n\\midrule\n\\multicolumn{12}{r}{Continued on next page} \\\\\n\\midrule\n\\endfoot\n\\bottomrule\n\\endlastfoot\n1A 0535+262 & O9.7IIIe & 8.60 & 181.44 & -2.64 & 0.53(0.0230) & 1.91(0.0850) & 2.13 & -2.03 & 12.76 & NaN & 20.00 \\\\\n1A 1118-61 & O9.5Ve & 11.59 & 292.50 & -0.89 & 0.33(0.0110) & 3.04(0.1060) & -5.57 & -0.51 & 13.18 & NaN & NaN \\\\\n1E 1145.1-6141 & B2Iae & 12.26 & 295.49 & -0.01 & 0.13(0.0100) & 7.89(0.6380) & -6.61 & 0.83 & 49.26 & 1.70 & 14.00 \\\\\n1ES 1210-64.6 & B5V & 13.98 & 298.89 & -2.30 & 0.26(0.0180) & 3.78(0.2530) & -5.96 & -0.38 & 6.14 & NaN & NaN \\\\\n1FGL J1018.6-5856 & O6V((f)) & 12.27 & 284.35 & -1.69 & 0.23(0.0100) & 4.40(0.1980) & -6.65 & -1.59 

In [24]:
# Function to round numerical values in a LaTeX table
def round_latex_table(file_path, decimal_places=2):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    rounded_lines = []
    for line in lines:
        if '&' in line and '\\hline' not in line:  # Identify data rows
            cells = line.split('&')
            rounded_cells = []
            for cell in cells:
                try:
                    # Attempt to round numeric values
                    rounded_cell = f"{round(float(cell.strip()), decimal_places)}"
                except ValueError:
                    # Leave non-numeric values unchanged
                    rounded_cell = cell.strip()
                rounded_cells.append(rounded_cell)
            rounded_lines.append(' & '.join(rounded_cells) + ' \\\\\n')
        else:
           rounded_lines.append(line)  # Keep non-data lines unchanged

    # Save the processed file
    processed_file_path = file_path.replace('.tex', '_rounded.tex')
    with open(processed_file_path, 'w') as file:
        file.writelines(rounded_lines)

    return processed_file_path


# Apply rounding to the uploaded LaTeX table
file_path = f"{home_files}/Tables/HMXBstars.tex"
processed_file = round_latex_table(file_path, decimal_places=2)
processed_file


'/home/karan/Documents/UvA/Thesis//Tables/HMXBstars_rounded.tex'